### This tutorial is divided into following sections:


1.   Implementing Decision Tree from scratch
2.   Implementing Decision Tree using scikit-learn
3.   Implementing Decision Tree using Tensorflow



# Implementing Decision Tree from scratch
### This section is divided into following parts:


1.   Gini Index
2.   Create Split
3.   Build a Tree
4.   Make a Precision
5.   Use Case



## Gini Index
The Gini index is the name of the cost function used to evaluate splits in the dataset.

A split in the dataset involves one input attribute and one value for that attribute. It can be used to divide training patterns into two groups of rows.

A Gini score gives an idea of how good a split is by how mixed the classes are in the two groups created by the split. A perfect separation results in a Gini score of 0, whereas the worst case split that results in 50/50 classes in each group result in a Gini score of 0.5 (for a 2 class problem).




The following is used to calculate gini index:

1. We calculate proprotion of classes in each group=(count of class)/size of group
2. gini index will be sum of are gini scores of class in each group = 1-sum(proportion*proportion)
3. we weight gini score of each group by size gini= (1-sum(proportion*proportin))

In [ ]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Calculate the Gini index for a split dataset
def gini_index(groups, classes):
	# count all samples at split point
	n_instances = float(sum([len(group) for group in groups]))
	# sum weighted Gini index for each group
	gini = 0.0
	for group in groups:
		size = float(len(group))
		# avoid divide by zero
		if size == 0:
			continue
		score = 0.0
		# score the group based on the score for each class
		for class_val in classes:
			p = [row[-1] for row in group].count(class_val) / size
			score += p * p
		# weight the group score by its relative size
		gini += (1.0 - score) * (size / n_instances)
	return gini

In [ ]:
# test Gini values
print(gini_index([[[1, 1], [1, 0]], [[1, 1], [1, 0]]], [0, 1]))
print(gini_index([[[1, 0], [1, 0]], [[1, 1], [1, 1]]], [0, 1]))

0.5
0.0


## Create A Split
Creating a split invloves three steps:
1. Calculating gini score
2. Splitting Dataset
3. Evaluating Split

We saw about gini score in previous section


### Splitting Dataset:
It is as simple as its name.We need index of attribute,dataset and value we are splitting with.

In [ ]:
# Split a dataset based on an attribute and an attribute value
def test_split(index, value, dataset):
	left, right = list(), list()
	for row in dataset:
		if row[index] < value:
			left.append(row)
		else:
			right.append(row)
	return left, right

### Evaluating split:
This is an exhaustive and greedy algorithm. We will use dictionary to store data as we can store data using names.We will store index of attribute,value of attribute and two groups of data split of best gini score

In [ ]:
# Select the best split point for a dataset
def get_split(dataset):
	class_values = list(set(row[-1] for row in dataset))
	b_index, b_value, b_score, b_groups = 999, 999, 999, None
	for index in range(len(dataset[0])-1):
		for row in dataset:
			groups = test_split(index, row[index], dataset)
			gini = gini_index(groups, class_values)
			if gini < b_score:
				b_index, b_value, b_score, b_groups = index, row[index], gini, groups
	return {'index':b_index, 'value':b_value, 'groups':b_groups}

## Build A Tree

Building a tree may be divided into 3 main parts:

1. Terminal Nodes.
2. Recursive Splitting.
3. Building a Tree.


### Termial Nodes:
We need to decide when to stop growing a tree

We can do that using the depth and the number of rows that the node is responsible for in the training dataset.


*   Maximum Tree Depth. This is the maximum number of nodes from the root node of the tree. Once a maximum depth of the tree is met, we must stop splitting adding new nodes. Deeper trees are more complex and are more likely to overfit the training data.
*   Minimum Node Records. This is the minimum number of training patterns that a given node is responsible for. Once at or below this minimum, we must stop splitting and adding new nodes. Nodes that account for too few training patterns are expected to be too specific and are likely to overfit the training data.


In [ ]:
# Create a terminal node value
def to_terminal(group):
	outcomes = [row[-1] for row in group]
	return max(set(outcomes), key=outcomes.count)

### Recursive Splitting:
Building a decision tree involves calling the above developed get_split() function over and over again on the groups created for each node.

This function is best explained in steps:

1. Firstly, the two groups of data split by the node are extracted for use and deleted from the node. As we work on these groups the node no longer requires access to these data.
2. Next, we check if either left or right group of rows is empty and if so we create a terminal node using what records we do have.
3. We then check if we have reached our maximum depth and if so we create a terminal node.
4. We then process the left child, creating a terminal node if the group of rows is too small, otherwise creating and adding the left node in a depth first fashion until the bottom of the tree is reached on this branch.
5. The right side is then processed in the same manner, as we rise back up the constructed tree to the root.


In [ ]:
#Create child splits for a node or make terminal
def split(node, max_depth, min_size, depth):
	left, right = node['groups']
	del(node['groups'])
	# check for a no split
	if not left or not right:
		node['left'] = node['right'] = to_terminal(left + right)
		return
	# check for max depth
	if depth >= max_depth:
		node['left'], node['right'] = to_terminal(left), to_terminal(right)
		return
	# process left child
	if len(left) <= min_size:
		node['left'] = to_terminal(left)
	else:
		node['left'] = get_split(left)
		split(node['left'], max_depth, min_size, depth+1)
	# process right child
	if len(right) <= min_size:
		node['right'] = to_terminal(right)
	else:
		node['right'] = get_split(right)
		split(node['right'], max_depth, min_size, depth+1)

### Building A Tree

In [ ]:
#Build a decision tree
def build_tree(train, max_depth, min_size):
	root = get_split(train)
	split(root, max_depth, min_size, 1)
	return root

In [ ]:
#Print a decision tree
def print_tree(node, depth=0):
	if isinstance(node, dict):
		print('%s[X%d < %.3f]' % ((depth*' ', (node['index']+1), node['value'])))
		print_tree(node['left'], depth+1)
		print_tree(node['right'], depth+1)
	else:
		print('%s[%s]' % ((depth*' ', node)))

dataset = [[2.7,1.2,0],
	[1.728,1.1,0],
	[3.678,2.8,0],
	[3.961,2.6,0],
	[2.999,2.2,0],
	[7.497,3.1,1],
	[9.203,3.3,1],
	[7.444,0.4,1],
	[10.12,3.2,1],
	[6.642,3.3,1]]
tree = build_tree(dataset, 1, 1)
print_tree(tree)

[X1 < 6.642]
 [0]
 [1]


In [ ]:
tree = build_tree(dataset,2,1)
print_tree(tree)

[X1 < 6.642]
 [X1 < 2.700]
  [0]
  [0]
 [X1 < 7.497]
  [1]
  [1]


In [ ]:
tree = build_tree(dataset,3,1)
print_tree(tree)

[X1 < 6.642]
 [X1 < 2.700]
  [0]
  [X1 < 2.700]
   [0]
   [0]
 [X1 < 7.497]
  [X1 < 7.444]
   [1]
   [1]
  [X1 < 7.497]
   [1]
   [1]


## Make A Prediction:

In [ ]:
#Make a prediction with a decision tree
def predict(node,dataset):
	results=np.array([0]*len(dataset))
  
	for i,row in enumerate(dataset):
		results[i]=get_prediction(node,row)
	
	return results

def get_prediction(node, row):
	if row[node['index']] < node['value']:
		if isinstance(node['left'], dict):
			return get_prediction(node['left'], row)
		else:
			return node['left']
	else:
		if isinstance(node['right'], dict):
			return get_prediction(node['right'], row)
		else:
			return node['right']

dataset=np.array([[2.7,1.2,0],
	[1.728,1.1,0],
	[3.678,2.8,0],
	[3.961,2.6,0],
	[2.999,2.2,0],
	[7.497,3.1,1],
	[9.203,3.3,1],
	[7.444,0.4,1],
	[10.12,3.2,1],
	[6.642,3.3,1]])

stump = {'index': 0, 'right': 1, 'value': 6.642287351, 'left': 0}
results=predict(stump,dataset)
results

array([0, 0, 0, 0, 0, 1, 1, 1, 1, 0])

In [ ]:
class DecisionTreeClassifier():

  def __init__(self,max_depth,depth=1,min_size=1):
    self.max_depth=max_depth
    self.depth=depth
    self.min_size=min_size

  def fit(self,x,y):
    self.x=x
    self.y=y

    self.train=np.concatenate((x,y),axis=1)
    self.build_tree(self.train,self.max_depth,self.min_size)

  def gini_index(self,groups, classes):
    n_instances = float(sum([len(group) for group in groups]))
    
    gini = 0.0
    for group in groups:
      size = float(len(group))
      if size == 0:
        continue
      score = 0.0
      for class_val in classes:
        p = [row[-1] for row in group].count(class_val) / size
        score += p * p
      gini += (1.0 - score) * (size / n_instances)
    return gini

  def test_split(self,index, value, dataset):
    left, right = list(), list()
    for row in dataset:
      if row[index] < value:
        left.append(row)
      else:
        right.append(row)
    return left, right

  def get_split(self,dataset):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    for index in range(len(dataset[0])-1):
      for row in dataset:
        groups = self.test_split(index, row[index], dataset)
        gini = self.gini_index(groups, class_values)
        if gini < b_score:
          b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}

  def to_terminal(self,group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

  def split(self,node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])
   
    if not left or not right:
      node['left'] = node['right'] = self.to_terminal(left + right)
      return
   
    if depth >= max_depth:
      node['left'], node['right'] = self.to_terminal(left), self.to_terminal(right)
      return
   
    if len(left) <= min_size:
      node['left'] = self.to_terminal(left)
    else:
      node['left'] = self.get_split(left)
      self.split(node['left'], max_depth, min_size, depth+1)
   
    if len(right) <= min_size:
      node['right'] = self.to_terminal(right)
    else:
      node['right'] = self.get_split(right)
      self.split(node['right'], max_depth, min_size, depth+1)
    

  def build_tree(self,train, max_depth, min_size):
    self.node = self.get_split(train)
    self.split(self.node, max_depth, min_size, 1)


  def predict(self,x):
    results=np.array([0]*len(x))

    for i,row in enumerate(x):
      results[i]=self._get_prediction(self.node,row)

    return results

  def _get_prediction(self, node, row):
    if row[node['index']] < node['value']:
      if isinstance(node['left'], dict):
        return self._get_prediction(node['left'], row)
      else:
        return node['left']
    else:
      if isinstance(node['right'], dict):
        return self._get_prediction(node['right'], row)
      else:
        return node['right']

    def printtree(self, depth=0):
	    if isinstance(self.node, dict):
		    print('%s[X%d < %.3f]' % ((depth*' ', (self.node['index']+1), self.node['value'])))
		    self.printtree(self.node['left'], depth+1)
		    self.printtree(self.node['right'], depth+1)
	    else:
		    print('%s[%s]' % ((depth*' ', self.node)))
  

In [ ]:
x=np.array([[2.7,1.2],
	[1.728,1.1],
	[3.678,2.8],
	[3.961,2.6],
	[2.999,2.2],
	[7.497,3.1],
	[9.203,3.3],
	[7.444,0.4],
	[10.12,3.2],
	[6.642,3.3]])
y=np.array([0,0,0,0,0,1,1,1,1,1])

In [ ]:
tree=DecisionTreeClassifier(max_depth=2)
tree.fit(x,y.reshape(-1,1))

In [ ]:
predict=tree.predict(x)
predict

array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1])

# Implementing Decision Tree using Scikit-Learn

Scikit-Learn has inbuilt DecisionTreeClassifier model which can be imported by using following line of code:

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# We can instantiate DecisionTreeClassifier as follows:
classifier=DecisionTreeClassifier()

## Paramters/Arguments
As we implemented DecisionTreeClassifier and it has some arguments or parameter such as max_depth,depth,min_size. Similarly scikit-learn's decision tree also has parameters.Some of them are as follows:

1. criterion : 'gini' or 'entropy' criteria used for information gain
2. max_depth : The maximum depth of the tree. If None, then nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples.

3. min_samples_split : int or float, default=2
 The minimum number of samples required to split an internal node:
If int, then consider min_samples_split as the minimum number.
If float, then min_samples_split is a fraction and ceil(min_samples_split * n_samples) are the minimum number of samples for each split.


4. min_samples_leaf : int or float, default=1
The minimum number of samples required to be at a leaf node. A split point at any depth will only be considered if it leaves at least min_samples_leaf training samples in each of the left and right branches. This may have the effect of smoothing the model, especially in regression.
If int, then consider min_samples_leaf as the minimum number.
If float, then min_samples_leaf is a fraction and ceil(min_samples_leaf * n_samples) are the minimum number of samples for each node. 

5. max_leaf_nodes : int, default=None
Grow a tree with max_leaf_nodes in best-first fashion. Best nodes are defined as relative reduction in impurity. If None then unlimited number of leaf nodes.

## Defining Classifier:

In [ ]:
#Let's define classifier with few of parameters
classifier=DecisionTreeClassifier(criterion='gini',
                                  max_depth=3,
                                  min_samples_split=2,
                                  min_samples_leaf=1,
                                  max_leaf_nodes=10)

## Training Classifier:

In [ ]:
#We can use fit() method to train classifier on our training set
classifier.fit(x,y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=3, max_features=None, max_leaf_nodes=10,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

## Using trained classifier for prediction:

In [ ]:
pred=classifier.predict(x)
pred

array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1])

# Implementing Decision Tree using Tensorflow 
Tensorflow also contains inbuilt model for decision tree.The following line of code can be used to import it.

In [ ]:
import tensorflow as tf
from tensorflow.estimator import BoostedTreesClassifier

## Defining Dataset parameters and building

**Before** instantiating classifier we must give names to our feature columns because BoostedTreeClassifier accepts dataset in such a way. 

In [ ]:
# Dataset parameters.
num_classes = 2 # Total classes: greater or equal to $23,000, or not (See notes below).
num_features = 2 # data features size.

# Training parameters.
max_steps = 200
batch_size = 2
learning_rate = 1.0
l1_regul = 0.0
l2_regul = 0.1

# GBDT parameters.
num_batches_per_layer = 4
num_trees = 1
max_depth = 4

## Building Input Function:

In [ ]:
# Build the input function.
train_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(
    x={'x': x}, y=y,
    batch_size=batch_size, num_epochs=None, shuffle=True)

test_train_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(
    x={'x': x}, y=y,
    batch_size=batch_size, num_epochs=1, shuffle=False)


# GBDT Models from TF Estimator requires 'feature_column' data format.
feature_columns = [tf.feature_column.numeric_column(key='x', shape=(num_features,))]

## Defining  Classifier:
Let's instantiate classifier:

In [ ]:
classifier = tf.estimator.BoostedTreesClassifier(
    n_batches_per_layer=num_batches_per_layer,
    feature_columns=feature_columns, 
    n_classes=num_classes,
    learning_rate=learning_rate, 
    n_trees=num_trees,
    max_depth=max_depth,
    l1_regularization=l1_regul, 
    l2_regularization=l2_regul
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp88kh1xis', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## Training Classifier:
Training our classifier using train_input_fn which contains our training data

In [ ]:
classifier.train(input_fn=train_input_fn,max_steps=max_steps)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp88kh1xis/model.ckpt.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has n

## Evaluating Classifier:
We can evaluate our model using evaluate attribute of classifier<br>
Note: Here we are evaluating classfir on training data

In [ ]:
evaluation=classifier.evaluate(test_train_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-08-12T04:58:41Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp88kh1xis/model.ckpt-160
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.39692s
INFO:tensorflow:Finished evaluation at 2020-08-12-04:58:42
INFO:tensorflow:Saving dict for global step 160: accuracy = 1.0, accuracy_baseline = 0.5, auc = 0.99999976, auc_precision_recall = 0.9999998, average_loss = 0.013007852, global_step = 160, label/mean = 0.5, loss = 0.013007852, precision = 1.0, prediction/mean = 0.4985648, recall = 1.0
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 160: /tmp/tmp88kh1xis/model.ckpt-160


In [ ]:
print(evaluation)

{'accuracy': 1.0, 'accuracy_baseline': 0.5, 'auc': 0.99999976, 'auc_precision_recall': 0.9999998, 'average_loss': 0.013007852, 'label/mean': 0.5, 'loss': 0.013007852, 'precision': 1.0, 'prediction/mean': 0.4985648, 'recall': 1.0, 'global_step': 160}


In [ ]:
#This gives us a generator object
pred=classifier.predict(test_train_input_fn)

#Converting generator objects to list then accessing class by using 'class_ids'
results=list(pred)
classes=[]
for result in results:
  classes.append(result['class_ids'][0])

print(classes)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp88kh1xis/model.ckpt-160
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
